# Secret Sharing by Adi Shamir

- Paper: "How to Share a Secret", CACM, 1979
- 2장짜리 논문
- CACM의 에디터는? Rivest

## (k, n) threshold scheme

- 키를 하나만 사용하는 건 위험한 일
  - 적어둔 종이가 사라지면? 누가 훔쳐가면?
  - 머리로 기억하게 했는데 그 사람이 죽으면?
  - 하드디스크에 보관했는데 해킹 당하면?
- n명에게 키를 나누어주고 k명이 모이면 풀 수 있도록

## 어떻게 하지?

- 여러 방법이 있겠지만, shamir가 제안한 간단한 방법은, polynomial interpolation
- 2차원 평면에 k개의 점이 있다면 k-1차 방정식을 복원할 수 있다

1. 랜덤 계수를 가지는 k-1차 방정식을 하나 만든다. q(x)라고 하자.
2. n개의 점을 얻는다. 예를 들면, $D_{1} = q(1)$, $D_{2} = q(2)$, ..., $D_{n} = q(n)$. 이 때, 꼭 1, 2, 3, .. 순서로 만들 필요는 없다. 어떤 숫자든 상관없다.
3. n개 좌표를 n명에게 나누어준다.
4. n명 중 k명이 모이면 이걸 이용해서 q(x)를 복원할 수 있다.

## q(x) 복원해서 뭐해?

- 정상적으로 참여한 k명은 모두 동일한 q(x)를 복원할 수 있을테니까 q(0)을 shared secret으로 쓰자.

## 그런데... 걱정되는 것이 있어


In [1]:
n, k = 11, 5

In [2]:
import sympy
import random

In [3]:
x = sympy.symbols("x")
a = sympy.IndexedBase("a")
u = sympy.Poly.from_list([a[i] for i in range(k)], gens=x)
u

Poly(a[0]*x**4 + a[1]*x**3 + a[2]*x**2 + a[3]*x + a[4], x, domain='EX')

In [4]:
u.subs(x, 1)  # q(1)

a[0] + a[1] + a[2] + a[3] + a[4]

In [5]:
u.subs(x, 2)  # q(2)

16*a[0] + 8*a[1] + 4*a[2] + 2*a[3] + a[4]

In [6]:
u.subs(x, 5)  # q(5)

625*a[0] + 125*a[1] + 25*a[2] + 5*a[3] + a[4]

In [7]:
u.subs(x, 2048)  # q(2048)

17592186044416*a[0] + 8589934592*a[1] + 4194304*a[2] + 2048*a[3] + a[4]

### 어떤 숫자든 쓸 수 있다며... ?! 안될 것 같은데?

- Shamir는 다 계획이 있구나.
- Modular arithmetic을 사용하는 것.
- 나머지 연산을 위해 소수를 하나 선택합시다.

In [8]:
p = sympy.randprime(1_000, 5_000)
p

4129

### 꼭 소수 써야 되나요?

- 나중에 나누기를 해야 할 일이 있습니다. $a / b = c$ (mod $p$)
- 그냥 나눠서 분수 만들면 mod $p$ 세상의 결과가 안나옵니다. **"닫혀 있다"** 표현 기억 나시나요?
- 역원을 곱하면 나누기가 됩니다.
- $b * x = 1$ mod $p$를 찾고 x를 곱해주면 되겠습니다.
  - 어떻게 찾죠? extended Euclidean algorithm이란 것이 있습니다.
  - 이 노트북에서는 스킵합니다.

In [12]:
inverses = [sympy.mod_inverse(x, 7) for x in range(1, 7)]
inverses

[1, 4, 5, 2, 3, 6]

In [13]:
[(x * inv) % 7 for x, inv in zip(range(1, 7), inverses)]

[1, 1, 1, 1, 1, 1]

In [26]:
set([sympy.mod_inverse(x, 8) for x in range(1, 8)])

ValueError: inverse of 2 (mod 8) does not exist

### 이제 다시 돌아와서, n개의 점을 실제로 뽑아봅시다.

In [9]:
coeffs = random.sample(range(1, p), k)
coeffs

[995, 765, 1951, 997, 1738]

In [10]:
q = sympy.Poly.from_list(coeffs, gens=x, modulus=p)
q

Poly(995*x**4 + 765*x**3 + 1951*x**2 + 997*x + 1738, x, modulus=4129)

In [14]:
D = [(i, q(i)) for i in range(1, n + 1)]
D

[(1, -1812),
 (2, 544),
 (3, -332),
 (4, 2044),
 (5, 875),
 (6, 857),
 (7, 1663),
 (8, -2057),
 (9, -31),
 (10, 603),
 (11, 71)]

In [15]:
k_choices = random.sample(D, k)
k_choices

[(10, 603), (8, -2057), (5, 875), (9, -31), (11, 71)]

### 근데 어떻게 방정식을 복원한다고요? 심지어 modular arithmetic까지 해야 하잖아요.

- 우리에겐 Lagrange(1736-1813) 형님이 있습니다.

#### Lagrange polynomial

- 점들의 집합이 ${(i_{n}, d_{n})}$이라고 가정합시다. 

$$L(x) = \sum_{j=0}^{k-1} d_{j}l_{j}(x)$$
where $$l_{j}(x) = \prod_{m=0, m \ne j}^{k-1} \frac{x-i_{m}}{i_{j}-i_{m}}$$

#### 아이고 머리야

- ~~알고 보면 별 것 아닙니다~~
- 점 2개는 너무 간단하니까 점이 3개였다고 해보겠습니다.

$$L(x) = \frac{(x-x_{2})(x-x_{3})}{(x_{1}-x_{2})(x_{1}-x_{3})}y_{1} + \frac{(x-x_{1})(x-x_{3})}{(x_{2}-x_{1})(x_{2}-x_{3})}y_{2} + \frac{(x-x_{1})(x-x_{2})}{(x_{3}-x_{1})(x_{3}-x_{2})}y_{3}$$
1. $L(x_{1}) = \frac{(x_{1}-x_{2})(x_{1}-x_{3})}{(x_{1}-x_{2})(x_{1}-x_{3})}y_{1} + 0 + 0 = y_{1}$
2. $L(x_{2}) = 0 + \frac{(x_{2}-x_{1})(x_{2}-x_{3})}{(x_{2}-x_{1})(x_{2}-x_{3})}y_{2} + 0 = y_{2}$
3. $L(x_{3}) = 0 + 0 + \frac{(x_{3}-x_{1})(x_{3}-x_{2})}{(x_{3}-x_{1})(x_{3}-x_{2})}y_{3} = y_{3}$

In [16]:
import functools as ft

In [17]:
def sigma(xs):
    return ft.reduce(lambda a, b: a + b, xs)

In [18]:
def prod(xs):
    return ft.reduce(lambda a, b: a * b, xs)

In [19]:
def mul(y, c):
    return sympy.Poly(y * c, modulus=p)

In [20]:
def div(y, c):
    return mul(y, sympy.mod_inverse(c, p))

In [21]:
lx = []
for i, d in k_choices:
    upper = prod([sympy.Poly(x - j, modulus=p) for j, _ in k_choices if i != j])
    lower = prod([i - j for j, _ in k_choices if i != j]) % p
    lx.append(div(upper, lower))

lx

[Poly(-413*x**4 + 1242*x**3 + 373*x**2 - 1030*x - 396, x, modulus=4129),
 Poly(-1147*x**4 - 1145*x**3 + 1122*x**2 + 1285*x - 275, x, modulus=4129),
 Poly(-562*x**4 + 711*x**3 - 1501*x**2 + 1344*x + 22, x, modulus=4129),
 Poly(-516*x**4 + 1028*x**3 + 569*x**2 - 1316*x + 550, x, modulus=4129),
 Poly(-1491*x**4 - 1836*x**3 - 563*x**2 - 283*x + 100, x, modulus=4129)]

In [22]:
L = sigma([mul(l, d) for (_, d), l in zip(k_choices, lx)])
L

Poly(995*x**4 + 765*x**3 + 1951*x**2 + 997*x + 1738, x, modulus=4129)

In [23]:
q

Poly(995*x**4 + 765*x**3 + 1951*x**2 + 997*x + 1738, x, modulus=4129)

In [27]:
L(0) == q(0)

True